## Race Predictions ##
### An exploration of data forcasting techniques so I don't forgot how data works in python ###

In [6]:
import requests
import pandas as pd

#### Load Data ####

For the data, I will be using the  'jolpi.ca F1 API' which can be found [here](https://api.jolpi.ca/ergast/).

At first I played around with the idea of loading and structuing all the data myself, although this was far less performant and would require the split into testing and training data manually,
So instead I will be using Pandas.

In [7]:
"""
# Get the bulk of the data as a python object
race_years = ['2024', '2023', '2022']
races = {}
for year in race_years:
    races[year] = {}

# Pandas can just do this all automatically but good to know how it roughly works :)
for year in race_years:
    
    request_limiter = 0
    results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100").json()["MRData"]
    for result in results["RaceTable"]["Races"]:
        races[year][result["raceName"]] = result["Results"]

    while(int(results["total"]) > request_limiter + 100):
        request_limiter += 100
        results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100&offset="+str(request_limiter)).json()["MRData"]
        for result in results["RaceTable"]["Races"]:  
            races[year][result["raceName"]] = result["Results"]
"""

'\n# Get the bulk of the data as a python object\nrace_years = [\'2024\', \'2023\', \'2022\']\nraces = {}\nfor year in race_years:\n    races[year] = {}\n\n# Pandas can just do this all automatically but good to know how it roughly works :)\nfor year in race_years:\n    \n    request_limiter = 0\n    results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100").json()["MRData"]\n    for result in results["RaceTable"]["Races"]:\n        races[year][result["raceName"]] = result["Results"]\n\n    while(int(results["total"]) > request_limiter + 100):\n        request_limiter += 100\n        results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100&offset="+str(request_limiter)).json()["MRData"]\n        for result in results["RaceTable"]["Races"]:  \n            races[year][result["raceName"]] = result["Results"]\n'

In [ ]:
# Since everything has to be tokenised in the dataframe, map drivers and constructors to ints

# For now structure it as race start -> race end
starting_positions = pd.DataFrame()
results = pd.DataFrame()

request_limiter = 0 
results = requests.get("http://api.jolpi.ca/ergast/f1/2024/results?limit=100").json()["MRData"]["RaceTable"]["Races"]
for race in range(len(results)):
    for position in range(len(results[race]["Results"])):
        res = results[race]["Results"][position]
        starting_positions = pd.concat([res["Driver"]["code"], res["Constructor"]["constructorId"], res["grid"]], ignore_index=True)

print(starting_positions.shape())

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [ ]:

for race in results:
    data = pd.DataFrame(race)
    df = pd.concat([df, data], ignore_index=True, fill_value=None)
while(int(results["total"]) > request_limiter + 100):
    results = requests.get("http://api.jolpi.ca/ergast/f1/2024/results?limit=100&offset="+str(request_limiter)).json()["MRData"]["Races"]
    for race in results:
        data = pd.DataFrame(race)
        df = pd.concat([df, data], ignore_index=True, fill_value=None)

# Check the final DataFrame
print(df.shape)

